In [1]:
#Real or Not? NLP with Disaster Tweets
#150716074 Ali Mertcan KARAMAN
#150715040 Ahmed AKSU
#150716073 Osman KORKAÇ

In [ ]:
# Common imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
from nltk import pos_tag, word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score, confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE
from keras.utils import np_utils
import nltk
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import words

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM

C:\Users\BresciaPC\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
#Loading data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.head() #the predictions made by our model on the test set will be put in an submission file like this one 

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [4]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
#An example of a NOT disaster tweet, target=0 represents a not disaster tweet
train[train["target"] == 0]["text"].values[3]

'My car is so fast'

In [6]:
##An example of a disaster tweet
train[train["target"] == 1]["text"].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [7]:
#columns of the training data
train.columns 

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [8]:
#An example of a one row with all columns
print(train.values[0])

[1 nan nan
 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all' 1]


In [9]:
test.columns #at the test set, we do not have the target, we have to predict whether they are a disaster or not disaster tweets

Index(['id', 'keyword', 'location', 'text'], dtype='object')

In [12]:
#Convert a collection of text documents to a matrix of token counts
#This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.
## let's get counts for the first 5 tweets in the data
count_vectorizer = CountVectorizer()
example_train_vectors = count_vectorizer.fit_transform(train["text"][0:5])
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space), to see them all
print(example_train_vectors.todense().shape)
print(example_train_vectors.todense())

(5, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 2 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 2 0 0 0 0 0 1 1 0 1 1 1 1 0 0
  0 2 0 0 0 1 0 0 0 0 0 2 0 0 0 1 0 0]
 [1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1]]


In [13]:
#Creating vectors for all of our tweets
train_vectors = count_vectorizer.fit_transform(train["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test["text"])

In [15]:
train_vectors.shape

(7613, 21637)

In [16]:
test_vectors.shape

(3263, 21637)

In [17]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [18]:
#Cleaning the dataset
def cleaning(x):
    quoteRemoval = x.replace('"','')
    spaceRemoval = re.sub("\s\s+" , " ", quoteRemoval)
    stringRemoval = spaceRemoval.strip()
    urlRemove = re.sub(r'http\S+', '', stringRemoval)
    contract = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in urlRemove.split()]) 
    specialChar = re.sub(r"[^a-zA-Z]+", ' ',urlRemove)
    stop = set(stopwords.words('english')) #to remove stop words like is,the,that etc
    stpwords = ' '.join([i for i in specialChar.lower().split() if i not in stop])
    stemmer = WordNetLemmatizer() # steaming is to remove all ing,ly etc ex: stem('playing') = play
    steming = ' '.join([stemmer.lemmatize(i) for i in stpwords.lower().split()])
    lengthy = ' '.join([i for i in steming.lower().split() if len(i.strip())>3])   
    return lengthy

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
#Apllying cleaning function to our datasets
train['cleaned'] = train['text'].apply(cleaning)
test['cleaned'] = test['text'].apply(cleaning)

In [21]:
#Creating vectors for all of cleaned tweets
train_vectors_cleaned = count_vectorizer.fit_transform(train["cleaned"])
test_vectors_cleaned = count_vectorizer.transform(test["cleaned"])
train_vectors_cleaned.shape

(7613, 13512)

In [22]:
test_vectors_cleaned.shape

(3263, 13512)

In [38]:
real_test=test['cleaned'].values.reshape(-1,1)
test.columns

Index(['id', 'keyword', 'location', 'text', 'cleaned'], dtype='object')

In [17]:
x = train['cleaned'].values.reshape(-1,1)
Y = train['target'].values
#TFidf: “Term Frequency – Inverse Document Frequency”
#Term Frequency: This summarizes how often a given word appears within a document.
#Inverse Document Frequency: This downscales words that appear a lot across documents.
TfIdf = TfidfVectorizer(max_features=13512,norm='l1',ngram_range=(1,1),stop_words='english',strip_accents='ascii',analyzer='word') #top 2000 words
tfmodel = TfIdf.fit(x.ravel()) # ravel is to 
#pickle.dump(tfmodel, open('TFIDF.pkl', 'wb'))
XX = tfmodel.transform(x.ravel())

In [18]:
sm = SMOTE(random_state=2,k_neighbors=15) #Creates virtual instances for a better training set
X1, Y1 = sm.fit_sample(XX, Y.ravel())

In [19]:
#Splitting train dataset into smaller parts and test it within itself for better evaluation
X_train, X_test, Y_train, Y_test = train_test_split(X1, Y1, test_size = 0.1,random_state=41)

In [20]:
#We used Tensorflow 2.0 
#Sequential model is provided within tensorflow.keras.models library
#It is a deepleraning model and we can creat and add multiple layers into this model easily
from tensorflow.keras.layers import LeakyReLU
model = Sequential()
model.add(Dense(128, kernel_initializer ='glorot_uniform',input_dim=X_train.shape[1]))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.50))
model.add(Dense(128, kernel_initializer ='glorot_uniform'))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.50))
model.add(Dense(1, kernel_initializer ='glorot_uniform', activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adamax',
              metrics=['acc',f1_m,precision_m, recall_m])

es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

model.summary()

model.fit(X_train, Y_train, batch_size = 12, nb_epoch = 100,callbacks=[es],validation_data=(X_test,Y_test))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1711872   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

In [45]:
YY = tfmodel.transform(real_test.ravel())
y_pre=model.predict(YY)
y_pre=np.round(y_pre).astype(int).reshape(3263)

In [54]:
print(y_pre[200:210])

[1 0 1 0 1 0 0 1 1 0]


In [53]:
print(test['text'][200:210])

200    Delhi government to provide free treatment to ...
201                        I'm going into a panic attack
202    End the Innovation Catch-22: Reduce the Attack...
203    @RealTwanBrown Yesterday I Had A Heat Attack ?...
204    TexasÛªs Gratuitously Cruel Attack On A Dying...
205    Cooper the Super Pooper. The hero dog who save...
206    Double ebony attack http://t.co/33V0RLlrKf #Bl...
207    Suspect in latest US theatre attack had psycho...
208    #People #Tilly the #Confused Cat Overcomes Hor...
209    To love you love you love you ...  Massive Att...
Name: text, dtype: object


In [56]:
sample_submission.columns

Index(['id', 'target'], dtype='object')

In [57]:
#Submission file is going to be used for submitting the competitions
sub=pd.DataFrame({'id':sample_submission['id'].values.tolist(),'target':y_pre})
sub.to_csv('submission.csv',index=False)

In [62]:
sub.head(9)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
